### Problem 1: two generators, single load, single bus optimization problem.

In [ ]:
### Problem formulation
# generator 1: "gas" (CCGT), marginal cost 70 EUR/MWh, capacity 50 MW
# generator 2: "coal" (hard coal), marginal cost 40 EUR/MWh, capacity 100 MW
# load: "DE" (Germany), 120 MW
# single time step

In [ ]:
import pypsa
import numpy as np
import pandas as pd
import linopy

### Create PyPSA network with components of the problem

In [ ]:
n = pypsa.Network()

In [ ]:
n

In [ ]:
n.add("Bus", "DE", v_nom=380)

In [ ]:
n.buses

In [ ]:
n.add(
    "Generator",
    "gas",
    bus="DE",
    p_nom_extendable=False,
    marginal_cost=70,  # €/MWh
    p_nom=50,  # MW
)
n.add(
    "Generator",
    "coal",
    bus="DE",
    p_nom_extendable=False,
    marginal_cost=40,  # €/MWh
    p_nom=100,  # MW
)

In [ ]:
n.generators

In [ ]:
# add load
n.add(
    "Load",
    "Germany",
    bus="DE",
    p_set=pd.Series([120], index=["now"]),  # MW
)

In [ ]:
n.loads_t.p_set

### Solve with PyPSA optimize module (use the default mathematical problem)

In [ ]:
n.optimize(solver_name="glpk")

In [ ]:
n.objective

In [ ]:
n.generators_t.p

### Explore pypsa model

In [ ]:
n.optimize.create_model()

In [ ]:
n.model.objective

In [ ]:
n.model.constraints

In [ ]:
n.model.constraints["Generator-fix-p-upper"]

In [ ]:
n.model.constraints["Bus-nodal_balance"]

### Let's write optimization problem manually (reproduce PyPSA model for our problem)

In [ ]:
# remove all constraints
for name in list(n.model.constraints):
    n.model.remove_constraints(name)

In [ ]:
n.model.constraints

In [ ]:
# remove objective
n.model.objective.expression = linopy.LinearExpression(None, model=n.model)

In [ ]:
n.model.objective

In [ ]:
n.model["Generator-p"]

In [ ]:
# objective expression
(
    n.model["Generator-p"].loc[:, "gas"] * n.generators.marginal_cost.loc["gas"]
    + n.model["Generator-p"].loc[:, "coal"] * n.generators.marginal_cost.loc["coal"]
)

In [ ]:
# set objective expression to the problem
n.model.objective.expression = (
    n.model["Generator-p"].loc[:, "gas"] * n.generators.marginal_cost.loc["gas"]
    + n.model["Generator-p"].loc[:, "coal"] * n.generators.marginal_cost.loc["coal"]
)

In [ ]:
# check it is there
n.model.objective

In [ ]:
n.model.add_constraints(
    n.model["Generator-p"].sum(dims="Generator") == n.loads_t.p_set["Germany"],
    name="nodal_balance",
)

In [ ]:
n.model.add_constraints(n.model["Generator-p"].loc[:, "gas"] >= 0, name="p_lower_gas")

In [ ]:
n.model.add_constraints(n.model["Generator-p"].loc[:, "coal"] >= 0, name="p_lower_coal")

In [ ]:
n.model.add_constraints(
    n.model["Generator-p"].loc[:, "gas"] <= n.generators.p_nom.loc["gas"],
    name="p_upper_gas",
)

In [ ]:
n.model.add_constraints(
    n.model["Generator-p"].loc[:, "coal"] <= n.generators.p_nom.loc["coal"],
    name="p_upper_coal",
)

In [ ]:
# check that we did a good job
n.model.constraints

In [ ]:
n.model.solve(solver_name="glpk")

In [ ]:
n.model.objective

In [ ]:
n.generators_t.p